## 1. Adjust path to your Dataset folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Adjust path to your Dataset folder
dataset_path = '/content/drive/MyDrive/Dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Install Dependencies

In [2]:
!pip install facenet-pytorch torch torchvision
!pip install sklearn opencv-python-headless matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## 3. Import Libraries

In [1]:
import os
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.neighbors import KNeighborsClassifier
import cv2
import gradio as gr

## 4. Initialize Models

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MTCNN for face detection & alignment
mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20, device=device)

# Inception Resnet (FaceNet) for embedding extraction
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

## 5. Load Dataset and Create Embeddings

In [5]:
embeddings = []
labels = []

for person_name in os.listdir(dataset_path):
    person_dir = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_dir):
        continue
    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        try:
            img = Image.open(img_path).convert('RGB')
        except:
            continue

        # Detect and crop the face
        face = mtcnn(img)
        if face is None:
            continue

        # Get embedding
        face = face.unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = resnet(face).cpu().numpy().flatten()

        embeddings.append(embedding)
        labels.append(person_name)

# Convert to arrays
X = np.array(embeddings)
y = np.array(labels)

## 6. Train a Classifier (KNN)

In [6]:
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X, y)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

## 7. Recognition Function

In [7]:
def recognize(image_path, threshold=0.8):
    img = Image.open(image_path).convert('RGB')
    face = mtcnn(img)
    if face is None:
        return 'No face detected'

    face = face.unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(face).cpu().numpy().flatten()

    dist, idx = knn.kneighbors([emb], n_neighbors=1)
    if dist[0][0] < threshold:
        return y[idx[0][0]]
    else:
        return 'Unknown'

## 8. Evaluate Model Accuracy

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split embeddings into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier on training set
knn_eval = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn_eval.fit(X_train, y_train)

# Predict on test set
y_pred = knn_eval.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9916666666666667

Classification Report:
                     precision    recall  f1-score   support

    Angelina Jolie       0.94      1.00      0.97        17
         Brad Pitt       1.00      1.00      1.00        15
 Denzel Washington       1.00      1.00      1.00        26
      Hugh Jackman       0.96      1.00      0.98        23
 Jennifer Lawrence       1.00      1.00      1.00        21
       Johnny Depp       1.00      1.00      1.00        14
      Kate Winslet       1.00      0.96      0.98        24
 Leonardo DiCaprio       1.00      1.00      1.00        21
         Megan Fox       1.00      0.94      0.97        18
   Natalie Portman       1.00      1.00      1.00        20
     Nicole Kidman       1.00      1.00      1.00        17
  Robert Downey Jr       1.00      1.00      1.00        16
    Sandra Bullock       1.00      1.00      1.00        21
Scarlett Johansson       0.98      1.00      0.99        42
        Tom Cruise       1.00      1.00      

## 9. Test the System on a Local File

In [8]:
test_img = '/content/test.png'  # Upload or specify path to test image
print("Identified as:", recognize(test_img))

Identified as: Nicole Kidman


## 10. Interactive Upload with Gradio

In [9]:
def recognize_gradio(img):
    # Convert numpy array to PIL Image if needed
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    # Detect face
    face = mtcnn(img)
    if face is None:
        return 'No face detected'

    # Get embedding
    face = face.unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(face).cpu().numpy().flatten()

    # Find nearest neighbor
    dist, idx = knn.kneighbors([emb], n_neighbors=1)
    if dist[0][0] < 0.8:
        return y[idx[0][0]]
    else:
        return 'Unknown'

# Create and launch the Gradio interface
iface = gr.Interface(
    fn=recognize_gradio,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Textbox(),
    title="Face Recognition",
    description="Upload an image to identify the person if they are in the dataset."
)
iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://560a4f91f8cce99cfb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://560a4f91f8cce99cfb.gradio.live
